<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Automatic Test Generation

In this project, we will build an automatic test generation and grading platform!
All we have to do is to provide a topic, the number of questions and the number of options for each question!
Based on this information, a suitable test is generated, presented to the user and graded automatically!

## Imports

In [2]:
import os
import openai
from dotenv import load_dotenv

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [3]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!
load_dotenv()

True

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Tell GPT how to generate the test

We tell GPT to create a multiple choiz quiz. Hence we define the topic, the number of possible answers as well as the number of questions.
To enable automatical grading later, GPT needs to incorporate the correct answer!


In [5]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
                 + f"Each question should have {num_possible_answers} options. "\
                 + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

In [18]:
create_test_prompt("Rust programming language", 4, 4)

"Create a multiple choice quiz on the topic of Rust programming language consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: '."

### OpenAI API Call
Let's use text-davinci-003 for normal text generation

In [23]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=create_test_prompt("Rust programming language", 4, 4),
                                            max_tokens=256,
                                            temperature=0.7)

In [24]:
response

<OpenAIObject text_completion id=cmpl-6sLEC737CDMJrIjMCzE9lRkNqdnE1 at 0x7fc3d122ab70> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nQ1. What is Rust?\nA. A programming language\nB. A type of metal\nC. A web browser\nD. An operating system\nCorrect Answer: A. A programming language\n\nQ2. What does Rust provide memory safety?\nA. it prevents memory leaks\nB. it uses garbage collection\nC. it does not allow pointers\nD. it provides data encapsulation\nCorrect Answer: C. it does not allow pointers\n\nQ3. What is the syntax of Rust similar to?\nA. JavaScript\nB. Python\nC. Java\nD. C++\nCorrect Answer: D. C++\n\nQ4. What is the main feature of Rust?\nA. Speed\nB. Security\nC. Modularity\nD. Reliability\nCorrect Answer: D. Reliability"
    }
  ],
  "created": 1678409332,
  "id": "cmpl-6sLEC737CDMJrIjMCzE9lRkNqdnE1",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 1

In [25]:
response["choices"][0]["text"]

'\n\nQ1. What is Rust?\nA. A programming language\nB. A type of metal\nC. A web browser\nD. An operating system\nCorrect Answer: A. A programming language\n\nQ2. What does Rust provide memory safety?\nA. it prevents memory leaks\nB. it uses garbage collection\nC. it does not allow pointers\nD. it provides data encapsulation\nCorrect Answer: C. it does not allow pointers\n\nQ3. What is the syntax of Rust similar to?\nA. JavaScript\nB. Python\nC. Java\nD. C++\nCorrect Answer: D. C++\n\nQ4. What is the main feature of Rust?\nA. Speed\nB. Security\nC. Modularity\nD. Reliability\nCorrect Answer: D. Reliability'

### Q/A Extraction

We now need to extract the questions and answers to present them to the students later

In [10]:
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view
 

In [11]:
create_student_view(response["choices"][0]["text"], 4)

{1: '\n\nQ1. What is the correct syntax used to create a function in Python?\nA. function my_func()\nB. def my_func()\nC. func my_func()\nD. create my_func()\n\n',
 2: '\nQ2. How do you access the last item in a list?\nA. list[-1]\nB. list[end]\nC. list[last]\nD. list[1]\n\n',
 3: '\nQ3. What is the correct syntax for a for loop in Python?\nA. for (x in list)\nB. for x in list\nC. for x list\nD. for each x in list\n\n',
 4: '\nQ4. What is the correct syntax to access the value of a dictionary key?\nA. dict[key]\nB. dict.key\nC. dict->key\nD. dict{key}\n\n'}

In [13]:
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers



In [14]:
extract_answers(response["choices"][0]["text"], 4)

{1: 'Correct Answer: B. def my_func()\n',
 2: 'Correct Answer: A. list[-1]\n',
 3: 'Correct Answer: B. for x in list\n',
 4: 'Correct Answer: A. dict[key]\n'}

### Exam simulation
Based on the extracted questions, we can now simulate the exam

In [16]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [17]:
student_answers = take(create_student_view(response["choices"][0]["text"], 4))



Q1. What is the correct syntax used to create a function in Python?
A. function my_func()
B. def my_func()
C. func my_func()
D. create my_func()



Q2. How do you access the last item in a list?
A. list[-1]
B. list[end]
C. list[last]
D. list[1]



Q3. What is the correct syntax for a for loop in Python?
A. for (x in list)
B. for x in list
C. for x list
D. for each x in list




KeyboardInterrupt: Interrupted by user

### Automatic Grading
Based on the student's answers and correct answers, we can now grade the test!

In [53]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    if grade < 60:
        passed = "Not passed!"
    else:
        passed = "Passed!"
    return f"{correct_answers} out of {len(answers)} correct! You achieved: {grade} % : {passed}"


In [54]:
grade(extract_answers(response["choices"][0]["text"], 4), student_answers)

'2 out of 4 correct! You achieved: 50.0 % : Not passed!'